In [2]:
!pip install transformers -q
!pip install datasets -q
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.5 MB/s eta 0:00:00


## 1. Business / Research Understanding Phase

## 2. Data Preparation/Understanding Phase

In [3]:
from datasets import get_dataset_config_names, load_dataset_builder

In [4]:
dataset_path = "amazon_reviews_multi"

configs = get_dataset_config_names(dataset_path)
print(configs)

['all_languages', 'de', 'en', 'es', 'fr', 'ja', 'zh']


In [5]:
ds_config = 'en'
ds_builder = load_dataset_builder(dataset_path, ds_config)

print(ds_builder.info.description)
print('dataset splits: ', ds_builder.info.splits)
print('dataset features:', ds_builder.info.features)

We provide an Amazon product reviews dataset for multilingual text classification. The dataset contains reviews in English, Japanese, German, French, Chinese and Spanish, collected between November 1, 2015 and November 1, 2019. Each record in the dataset contains the review text, the review title, the star rating, an anonymized reviewer ID, an anonymized product ID and the coarse-grained product category (e.g. ‘books’, ‘appliances’, etc.) The corpus is balanced across stars, so each star rating constitutes 20% of the reviews in each language.

For each language, there are 200,000, 5,000 and 5,000 reviews in the training, development and test sets respectively. The maximum number of reviews per reviewer is 20 and the maximum number of reviews per product is 20. All reviews are truncated after 2,000 characters, and all reviews are at least 20 characters long.

Note that the language of a review does not necessarily match the language of its marketplace (e.g. reviews from amazon.de are pr

In [6]:
from datasets import load_dataset

dataset = load_dataset(dataset_path, ds_config)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

## 3. Exploratory Data Analysis Phase

In [8]:
import wandb

wandb_apikey = 'd0957427fd4084b53a15acbf9e3abc958c23af93'
wandb.login(key=wandb_apikey)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
run = wandb.init(project="sentiment-analysis", job_type='upload-dataset')
artifact = wandb.Artifact(name='amazon_reviews_english', type='dataset')


In [21]:
files = [dataset.cache_files[key][0]['filename'] for key in dataset.cache_files]

for path in files:
    artifact.add_file(local_path=path)


In [22]:
features = ds_builder.info.features

for key, value in features.items():
    print(key, value)

review_id Value(dtype='string', id=None)
product_id Value(dtype='string', id=None)
reviewer_id Value(dtype='string', id=None)
stars Value(dtype='int32', id=None)
review_body Value(dtype='string', id=None)
review_title Value(dtype='string', id=None)
language Value(dtype='string', id=None)
product_category Value(dtype='string', id=None)


In [23]:
import pandas as pd

df = pd.DataFrame()
for key in dataset:
    df_tmp = dataset[key].to_pandas()
    df_tmp['split'] = key
    df = pd.concat([df, df_tmp])

df.shape

(210000, 9)

In [24]:
df_sample = df[df['split'] == 'train'].sample(1000)

table = wandb.Table(columns=list(df_sample.columns), data=df_sample)
artifact.add(table, "eda_table_sample")

ArtifactManifestEntry(path='eda_table_sample.table.json', digest='vEFJF7Q3GzpYyadFHJI1eQ==', ref=None, birth_artifact_id=None, size=300703, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmppne6jotk')

In [25]:
table = wandb.Table(columns=list(df.columns), data=df)
wandb.Table.MAX_ROWS = df.shape[0] + 1000
artifact.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='0jZvFxk60FhjfQMBbjZXQQ==', ref=None, birth_artifact_id=None, size=60790671, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmpls1a1bn4')

In [ ]:
run.log_artifact(artifact)
run.finish()

## 4. Setup Phase

In [33]:
dataset_path = "amazon_reviews_multi"
ds_config = 'en'

text_col = 'review_body'
label_col = 'stars'


In [40]:
from datasets import load_dataset

raw_datasets = load_dataset(dataset_path, ds_config)

  0%|          | 0/3 [00:00<?, ?it/s]

In [41]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
from transformers import DataCollatorWithPadding

def tokenization(example):
    return tokenizer(example[text_col], padding=True, truncation=True, 
                     max_length=128, )

tokenized_datasets = raw_datasets.map(tokenization, batched=True, input_columns=[text_col, label_col])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [58]:
tokenized_datasets["train"]

Dataset({
    features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200000
})

In [57]:
# data_collator(
tokenized_datasets["train"].select_columns(['input_ids', 'token_type_ids', 'attention_mask', label_col])

{'stars': [1, 1, 1, 1, 1]}

In [45]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [71]:

number_of_classes = len(pd.unique(raw_datasets['train']['stars']))

In [72]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=number_of_classes)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_

In [ ]:
model.config

In [64]:
ds = tokenized_datasets["train"].select_columns(['input_ids', 'token_type_ids', 'attention_mask', label_col])

ds = ds.rename_column(label_col, 'labels')
ds

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200000
})

In [65]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=ds,
    # eval_dataset=tokenized_datasets["validation"].select_columns(['input_ids', 'token_type_ids', 'attention_mask', label_col]),
    # data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

## 5. Modeling Phase

## 6. Evaluation Phase

## 7. Deployment Phase